Modifier certaines valeurs d'une colonne

In [23]:
import pandas as pd
import random
import numpy as np
dates = pd.date_range("2020-01"
, end="2022-06"
, freq="W-MON")
categories = "trottinette vélo voiture marche tram bus train".split()
true_false = (dates.month > 3) * (dates.month < 7)
delay = (pd.Timestamp('now') - dates).round('D')
big_df = pd.DataFrame(
{'mobility': pd.Categorical(random.choices(categories, k=dates.size)),
'start': 0,
'stop': 0.1 * np.array(random.choices(list(range(100)), k=dates.size)),
'delay':delay,
'date':dates,
'target':true_false})
ventes = pd.read_csv("sales.csv")

big_df['couleur'] = pd.Series({0:'green', 25:'yellow', 100:'red'})
big_df.couleur.fillna('noire', inplace=True)
big_df.loc[big_df.couleur=='noire', 'couleur'] = 'black'
big_df.couleur.replace({'noire':"black"}) # attention nouvelle série
big_df[big_df.couleur=='black'].stop.cumsum()
big_df

,mobility,start,stop,delay,date,target,couleur
0,tram,0,7.3,1123 days,2020-01-06,False,green
1,bus,0,6.4,1116 days,2020-01-13,False,black
2,bus,0,6.0,1109 days,2020-01-20,False,black
3,tram,0,3.8,1102 days,2020-01-27,False,black
4,voiture,0,7.6,1095 days,2020-02-03,False,black
...,...,...,...,...,...,...,...
121,voiture,0,2.4,276 days,2022-05-02,True,black
122,vélo,0,5.5,269 days,2022-05-09,True,black
123,voiture,0,1.3,262 days,2022-05-16,True,black
124,train,0,8.2,255 days,2022-05-23,True,black


Corrélation et covariance et autres

In [24]:
analyse = ventes.pct_change()
analyse.columns
analyse.TV.corr(analyse.Sales)
analyse.corrwith(analyse.Sales)
analyse.corr()
analyse.cov()

,TV,Radio,Newspaper,Sales
TV,730.510398,0.433840,9.152641,21.202444
Radio,0.433840,46.509113,-0.635461,1.501722
Newspaper,9.152641,-0.635461,14.282074,0.591095
Sales,21.202444,1.501722,0.591095,1.007163


Quel est dans ventes le numéro de l'enregistrement classé au 7ème rang si on trie par
ordre décroissant en fonction de la colonne "TV" ?
Classez big_df en fonction de la colonne "mobility", quelle est la valeur de la variable
"stop" pour le premier enregistrement ?
Utilisez sort_index sur les colonnes de la table big_df , quel est l'ordre des colonnes ?

In [25]:
ventes.sort_values(by="TV", ascending=False).iloc[6]
big_df.sort_values(by="mobility").iloc[0].stop
big_df.sort_index() # ordre : mobility, start, stop, delay, date,target,couleur
big_df.sort_index(axis=1) # par ordre alphabétique 

,couleur,date,delay,mobility,start,stop,target
0,green,2020-01-06,1123 days,tram,0,7.3,False
1,black,2020-01-13,1116 days,bus,0,6.4,False
2,black,2020-01-20,1109 days,bus,0,6.0,False
3,black,2020-01-27,1102 days,tram,0,3.8,False
4,black,2020-02-03,1095 days,voiture,0,7.6,False
...,...,...,...,...,...,...,...
121,black,2022-05-02,276 days,voiture,0,2.4,True
122,black,2022-05-09,269 days,vélo,0,5.5,True
123,black,2022-05-16,262 days,voiture,0,1.3,True
124,black,2022-05-23,255 days,train,0,8.2,True


détection et filtre des valeurs aberrantes

In [26]:
np.random.seed(42) # graine pour tirage aléatoire controlé
valeurs = np.random.randn(100, 4) # matrice 100x4 de valeurs aléatoire
df = pd.DataFrame(valeurs, columns=[x*3 for x in 'ABCD'])
ages = random.choices(list(range(16,125)), k=100)
df['age'] = ages
df.columns

small = df.drop('age', axis=1)
small[np.abs(small)>2.5]= np.sign(small)*2.5
small

,AAA,BBB,CCC,DDD
0,0.496714,-0.138264,0.647689,1.523030
1,-0.234153,-0.234137,1.579213,0.767435
2,-0.469474,0.542560,-0.463418,-0.465730
3,0.241962,-1.913280,-1.724918,-0.562288
4,-1.012831,0.314247,-0.908024,-1.412304
...,...,...,...,...
95,-0.839722,-0.599393,-2.123896,-0.525755
96,-0.759133,0.150394,0.341756,1.876171
97,0.950424,-0.576904,-0.898415,0.491919
98,-1.320233,1.831459,1.179440,-0.469176


Dans le fichier ventes , ajoutez une colonne "pub" qui a pour valeur la moyenne des 3
médias. Construisez alors une colonne "cout" qui vaudra "gros" si pub est dans le dernier
quartile, "micro" si pub est dans le premier quartile et "moyen" sinon

In [27]:
ventes['pub'] = (ventes.TV+ventes.Newspaper+ventes.Sales)/3
ventes['pub'] = np.mean(ventes[['TV', 'Newspaper', 'Sales']], axis=1)#autre
ventes.describe()

q1 = ventes['pub'].quantile(0.25)
q3 = ventes['pub'].quantile(0.75)

ventes.loc[ventes['pub'] >= q3, "cout"] = "gros"
ventes.loc[ventes['pub'] <= q1, "cout"] = "micro"
ventes.loc[(ventes['pub'] > q1) & (ventes['pub'] < q3), "cout"] = "moyen"

ventes

,TV,Radio,Newspaper,Sales,pub,cout
0,230.1,37.8,69.2,22.1,107.133333,gros
1,44.5,39.3,45.1,10.4,33.333333,micro
2,17.2,45.9,69.3,12.0,32.833333,micro
3,151.5,41.3,58.5,16.5,75.500000,moyen
4,180.8,10.8,58.4,17.9,85.700000,moyen
...,...,...,...,...,...,...
195,38.2,3.7,13.8,7.6,19.866667,micro
196,94.2,4.9,8.1,14.0,38.766667,moyen
197,177.0,9.3,6.4,14.8,66.066667,moyen
198,283.6,42.0,66.2,25.5,125.100000,gros


discrétisation et regroupement


In [28]:
bins = [16, 18, 25, 35, 60, 100]
tags = "ados jeunes juniors adultes séniors".split()
cat1 = pd.cut(ages, bins)
cat2 = pd.cut(ages, bins, right=False, labels=tags)

cat3 = pd.cut(ages, 5, precision=1) # nombre de chiffres décimaux
cat3.value_counts()

cat4 = pd.qcut(ages, 4)
cat4.value_counts()
cat5 = pd.qcut(ages, [0, .1, .4, .6, .8, 1.], labels=tags)
cat5.describe()

pd.get_dummies(cat1)
pd.get_dummies(cat5, prefix="cat")

,cat_ados,cat_jeunes,cat_juniors,cat_adultes,cat_séniors
0,1,0,0,0,0
1,0,0,1,0,0
2,0,0,0,0,1
3,0,0,1,0,0
4,0,1,0,0,0
...,...,...,...,...,...
95,1,0,0,0,0
96,0,0,1,0,0
97,1,0,0,0,0
98,1,0,0,0,0


Opérations d'ajout et de jointures concat , merge


In [29]:
s1 = pd.Series([0,1], index=['a','b'])
s2 = pd.Series([2,3,4], index=list('cde'))
s3 = pd.Series([5,6], index=list('xy'))
pd.concat([s1,s2,s3]) # concat lignes
pd.concat([s1,s2,s3], axis=1) # concat colonnes
s4 = pd.concat([s1*5, s3])
pd.concat([s1, s4], axis=1)
pd.concat([s1, s4], axis=1, join='inner') # intersection lignes
pd.concat([s1, s4], axis=1, join='outer') # comportement par défaut

,0,1
a,0.0,0
b,1.0,5
x,NaN,5
y,NaN,6


In [38]:
t1 = pd.DataFrame({'a': [1,2],'b':[3,4]})
t2 = pd.DataFrame({'a': [11,12,13],'c':[23,24,25]})
pd.concat([t1,t2]) # concaténation ligne, union colonne
pd.concat([t1,t2], axis=1) # concatenation colonne, union ligne
pd.concat([t1,t2], join='inner') # concat lignes, intersection colonnes
pd.concat([t1,t2], axis=1, join='inner') # concat colonnes, intersection lignes

pd.merge(t1, t2, left_index=True, right_index=True)
pd.merge(t1, t2, how='outer', left_index=True, right_index=True)

,a_x,b,a_y,c
0,1.0,3.0,11,23
1,2.0,4.0,12,24
2,NaN,NaN,13,25


Application

In [31]:
coeff = pd.Series([1,1,2,2,.5], index=['ecoge',
'scico',
'math',
'info',
'option'])
maths = pd.DataFrame({'userID':[8,4,2,7,1,5],
'eval':[12,10,5,8,13,17]})
info = pd.DataFrame({'userID':[1,2,4,3,5,7],
'eval':[10,5,8,13,15,11]})
ecoge = pd.DataFrame({'userID':[1,4,3,8],
'eval': [14, 17, 12, 10]})
scico = pd.DataFrame({'userID': [6,5,2,7],
'eval': [17, 13, 9, 11]} )
option = pd.DataFrame({'userID': range(1,9),
'eval': [14]*8 })
report = pd.DataFrame({'userID': [6,8,6],
'note': [10, 13, 12],
'matiere':['maths','info','info']})
coeff.describe()
maths.describe()
info.describe()
ecoge.describe()
option.describe()
report.describe()

,userID,note
count,3.000000,3.000000
mean,6.666667,11.666667
std,1.154701,1.527525
min,6.000000,10.000000
25%,6.000000,11.000000
50%,6.000000,12.000000
75%,7.000000,12.500000
max,8.000000,13.000000


Vous devez sortir le classement d'une promotion, déterminer quels sont ceux qui valident
ou pas leur année. Pour valider son année, il faut avoir la moyenne sur l'ensemble des
matières. On suppose qu'une personne inscrite en "ecoge" n'est pas inscrite en "scico" et
réciproquement, par ailleurs toutes les autres matières sont obligatoires, une note
manquante est un 0. Les reports, sont les notes attribuées à une session antérieure.

In [46]:
df = pd.merge(maths, info, on='userID', how='outer', suffixes=('_maths', '_info'))
df = pd.merge(df, ecoge, on='userID', how='outer')
df = pd.merge(df, scico, on='userID', how='outer', suffixes=('_ecoge', '_scico'))
df = pd.merge(df, option, on='userID', how='outer')

df.fillna(0, inplace=True)

df['eval_maths'] = report['note'][(report['matiere'] == 'maths') & (report['userID'] == df['userID'])]
# df['mean'] = (df['eval_maths'] * coeff['math']
#         + df['eval_info'] * coeff['info']
#         + df['eval_ecoge'] * coeff['ecoge']
#         + df['eval_scico'] * coeff['scico']
#         + df['eval'] * coeff['option']) / (coeff.sum() - coeff['scico'])

# df['valid'] = (df['mean'] >= 10)
df


ValueError: Can only compare identically-labeled Series objects